<a href="https://colab.research.google.com/github/shubhamksingh1/WordPrediction_and_Autosuggestion/blob/main/WordPrediction/BasicModel/Word_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
%tensorflow_version 1.x

In [ ]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 
import numpy as np 

Using TensorFlow backend.


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
tokenizer = Tokenizer()

In [ ]:
def dataset_preparation(data):

	# basic cleanup
	corpus = data.lower().split("\n")

	# tokenization	
	tokenizer.fit_on_texts(corpus)
	total_words = len(tokenizer.word_index) + 1

	# create input sequences using list of tokens
	input_sequences = []
	for line in corpus:
		token_list = tokenizer.texts_to_sequences([line])[0]
		for i in range(1, len(token_list)):
			n_gram_sequence = token_list[:i+1]
			input_sequences.append(n_gram_sequence)

	# pad sequences 
	max_sequence_len = max([len(x) for x in input_sequences])
	input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

	# create predictors and label
	predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
	label = ku.to_categorical(label, num_classes=total_words)

	return predictors, label, max_sequence_len, total_words


In [ ]:
def create_model(predictors, label, max_sequence_len, total_words):
	
	model = Sequential()
	model.add(Embedding(total_words, 10, input_length=max_sequence_len-1))
	model.add(LSTM(150, return_sequences = True))
	# model.add(Dropout(0.2))
	model.add(LSTM(100))
	model.add(Dense(total_words, activation='softmax'))

	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')
	model.fit(predictors, label, epochs=100, verbose=1, callbacks=[earlystop])
	print (model.summary())
	return model 

In [ ]:
def generate_text(seed_text, next_words, max_sequence_len):
	for _ in range(next_words):
		token_list = tokenizer.texts_to_sequences([seed_text])[0]
		token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
		predicted = model.predict_classes(token_list, verbose=0)
		
		output_word = ""
		for word, index in tokenizer.word_index.items():
			if index == predicted:
				output_word = word
				break
		seed_text += " " + output_word
	return seed_text

In [ ]:
data = open('/content/drive/My Drive/WordPrediction/BajajHealthInsurance.txt').read()

In [ ]:
predictors, label, max_sequence_len, total_words = dataset_preparation(data)

In [ ]:
predictors

array([[  0,   0,   0, ...,   0,   0,   3],
       [  0,   0,   0, ...,   0,   3,   2],
       [  0,   0,   0, ...,   3,   2,  48],
       ...,
       [  0,   0,   0, ..., 499,   1, 500],
       [  0,   0,   0, ...,   1, 500,   5],
       [  0,   0,   0, ..., 500,   5,   1]], dtype=int32)

In [ ]:
label

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
max_sequence_len

44

In [ ]:
total_words

501

In [ ]:
model = create_model(predictors, label, max_sequence_len, total_words)

Epoch 1/100
1381/1381 [==============================] - 6s 5ms/step - loss: 5.9253 - acc: 0.0355
Epoch 2/100
1381/1381 [==============================] - 5s 4ms/step - loss: 5.5070 - acc: 0.0478
Epoch 3/100
1381/1381 [==============================] - 5s 4ms/step - loss: 5.4521 - acc: 0.0420
Epoch 4/100
1381/1381 [==============================] - 5s 4ms/step - loss: 5.4378 - acc: 0.0471
Epoch 5/100
1381/1381 [==============================] - 5s 4ms/step - loss: 5.4279 - acc: 0.0471
Epoch 6/100
1381/1381 [==============================] - 5s 4ms/step - loss: 5.4244 - acc: 0.0471
Epoch 7/100
1381/1381 [==============================] - 5s 4ms/step - loss: 5.4197 - acc: 0.0471
Epoch 8/100
1381/1381 [==============================] - 5s 4ms/step - loss: 5.4132 - acc: 0.0471
Epoch 9/100
1381/1381 [==============================] - 5s 4ms/step - loss: 5.4056 - acc: 0.0471
Epoch 10/100
1381/1381 [==============================] - 5s 4ms/step - loss: 5.3792 - acc: 0.0514
Epoch 11/100
1381/1

In [ ]:
print (generate_text("accident", 2, max_sequence_len))

accident health insurance


In [ ]:
from keras.models import load_model

model.save('/content/drive/My Drive/WordPrediction/modelV1.h5')  # creates a HDF5 file 'my_model.h5'
del model  # deletes the existing model

# returns a compiled model
# identical to the previous one


OSError: ignored

In [ ]:
model = load_model('/content/drive/My Drive/WordPrediction/modelV1.h5')

In [ ]:
print (generate_text("accident", 2, max_sequence_len))

accident health insurance


In [ ]:
print (generate_text("individual", 2, max_sequence_len))

individual health insurance


In [ ]:
print (generate_text("hospital", 1, max_sequence_len))

hospital to
